In [1]:
import cheetah
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense
import wandb
from wandb.keras import WandbCallback

initializing ocelot...


In [2]:
sweep_config = {
    "name": "my-sweep",
    "method": "random",
    "parameters": {
        "n_samples": {
            "values": [300, 5000, 10000]
        },
        "epochs": {
            "values": [42, 200, 1000]
        },
        "learning_rate": {
            "min": 0.0001,
            "max": 0.1
        },
        "batch_size": {
            "values": [32, 64, 128]
        },
        "layer_width": {
            "values": [11, 32, 400]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="quadrupole-surrogate", entity="msk-ipc")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: c1m1k9ju
Sweep URL: https://wandb.ai/msk-ipc/uncategorized/sweeps/c1m1k9ju


In [3]:
wandb.init(
    project="quadrupole-surrogate",
    entity="msk-ipc"
)
config = wandb.config

wandb: Currently logged in as: msk-ipc (use `wandb login --relogin` to force relogin)


### Generate input dataset
Each sample is array of beam parameters, k1 and length.

In [4]:
def generate_data(n_samples):
    parameter_keys = ['energy','mu_x','mu_xp','mu_y','mu_yp','sigma_p','sigma_s','sigma_x','sigma_xp','sigma_y','sigma_yp']

    # Input data
    parameters = [{
        "mu_x": np.random.uniform(-1e-3, 1e-3),
        "mu_y": np.random.uniform(-1e-3, 1e-3),
        "mu_xp": np.random.uniform(-1e-4, 1e-4),
        "mu_yp": np.random.uniform(-1e-4, 1e-4),
        "sigma_x": np.random.uniform(1e-5, 5e-4),
        "sigma_y": np.random.uniform(1e-5, 5e-4),
        "sigma_xp": np.random.uniform(1e-6, 5e-5),
        "sigma_yp": np.random.uniform(1e-6, 5e-5),
        "sigma_s": np.random.uniform(1e-6, 5e-5),
        "sigma_p": np.random.uniform(1e-4, 1e-3),
        "energy": np.random.uniform(80e6, 160e6)
    } for _ in range(n_samples)]

    beams = [cheetah.ParameterBeam.from_parameters(**p) for p in parameters]

    X1 = np.array([[b.parameters[k] for k in parameter_keys] for b in beams])

    X0 = np.array([[np.random.uniform(0.1, 0.3), np.random.uniform(-15.0, 15.0)] for _ in range(n_samples)])

    X = np.hstack([X0, X1])

    # Output data
    y = []
    for incoming, x in zip(beams, X0):
        quadrupole = cheetah.Quadrupole(length=x[0], k1=x[1])
        outgoing = quadrupole(incoming)
        y.append(outgoing)

    y = np.array([[b.parameters[k] for k in parameter_keys] for b in y])

    return X, y

### Train ANN
Not comment needed.

In [5]:
def train():
    with wandb.init(project="quadrupole-surrogate", entity="msk-ipc") as run:
        config = wandb.config

        # Prepare data
        X, y = generate_data(config.n_samples)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)
        X_scaler = StandardScaler()
        X_train_scaled = X_scaler.fit_transform(X_train)
        X_test_scaled = X_scaler.transform(X_test)
        y_scaler = StandardScaler()
        y_train_scaled = y_scaler.fit_transform(y_train)
        y_test_scaled = y_scaler.transform(y_test)

        # Make model
        model = keras.Sequential([
            Dense(config.layer_width, activation="relu"),
            Dense(config.layer_width, activation="relu"),
            Dense(11)
        ])
        model.compile(loss="mse", optimizer="adam", metrics=["mae"])

        # Train
        model.fit(
            X_train_scaled,
            y_train_scaled,
            batch_size=config.batch_size,
            epochs=config.epochs,
            validation_data=(X_test_scaled,y_test_scaled),
            callbacks=[WandbCallback()]
        )

In [6]:
wandb.agent(sweep_id, train, count=5)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: b4gf89j7 with config:
wandb: 	batch_size: 128
wandb: 	epochs: 1000
wandb: 	layer_width: 11
wandb: 	learning_rate: 0.016561578871106257
wandb: 	n_samples: 300
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


2022-03-29 15:58:53.726147: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/1000
2/2 [==============================] - 7s 2s/step - loss: 1.1368 - mae: 0.8672 - val_loss: 1.1536 - val_mae: 0.8562 - _timestamp: 1648562350.0000 - _runtime: 28.0000
Epoch 2/1000
2/2 [==============================] - 0s 135ms/step - loss: 1.1203 - mae: 0.8608 - val_loss: 1.1429 - val_mae: 0.8519 - _timestamp: 1648562350.0000 - _runtime: 28.0000
Epoch 3/1000
2/2 [==============================] - 0s 311ms/step - loss: 1.1039 - mae: 0.8547 - val_loss: 1.1330 - val_mae: 0.8479 - _timestamp: 1648562351.0000 - _runtime: 29.0000
Epoch 4/1000
2/2 [==============================] - 0s 301ms/step - loss: 1.0900 - mae: 0.8493 - val_loss: 1.1236 - val_mae: 0.8442 - _timestamp: 1648562351.0000 - _runtime: 29.0000
Epoch 5/1000
2/2 [==============================] - 1s 490ms/step - loss: 1.0763 - mae: 0.8440 - val_loss: 1.1149 - val_mae: 0.8409 - _timestamp: 1648562351.0000 - _runtime: 29.0000
Epoch 6/1000
2/2 [==============================] - 0s 151ms/step - loss: 1.0644 - mae: 0.839